## Data preprocess

In [1]:
import pandas as pd
import jieba
import logging
from gensim.models import word2vec

# set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(levelname)s] <%(processName)s> (%(threadName)s) %(message)s')
logger = logging.getLogger()

# init variables
train_data_path = "data/training/sentiment_analysis_trainingset.csv"
validate_data_path = "data/validation/sentiment_analysis_validationset.csv"
test_data_path = "data/test_a/sentiment_analysis_testa.csv"
test_data_predict_out_path = "data/test_a/sentiment_analysis_testa_prediction.csv"
stop_world_dpath = "data/stop_words.txt"

# loading data
def load_data_from_csv(file_name, header=0, encoding="utf-8"):
    data_df = pd.read_csv(file_name, header=header, encoding=encoding)
    return data_df

# segment words 
def seg_words(contents):
    contents_segs = list()
    for content in contents:
        segs = jieba.lcut(content)
        content = " ".join(segs)
        content = content.replace("\n", "")
        contents_segs.append(content)
    return contents_segs

# get stop words
def stop_words():
    # loading Chinese stop worlds
    stpwrd_dic = open(stop_world_dpath, 'rb')
    stpwrd_content = stpwrd_dic.read().decode('utf8')
    # Decode and put into a list
    stpwrdlst = stpwrd_content.splitlines()
    stpwrdlst[0] = ','
    stpwrd_dic.close()
    return stpwrdlst

c:\users\tracy\anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [15]:
# load train and validation data
train_data_df = load_data_from_csv(train_data_path)
validate_data_df = load_data_from_csv(validate_data_path)
test_data_df = load_data_from_csv(test_data_path)
# loading all the comment
content_train = train_data_df.iloc[:, 1]
content_val = validate_data_df.iloc[:, 1]
content_test = test_data_df.iloc[:, 1]

In [3]:
# adding suggestion
jieba.suggest_freq('大众点评', True)
jieba.suggest_freq('大众点评网', True)
jieba.suggest_freq('港囧', True)

Building prefix dict from the default dictionary ...
2018-10-01 21:10:17,691 [DEBUG] <MainProcess> (MainThread) Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\tracy\AppData\Local\Temp\jieba.cache
2018-10-01 21:10:17,693 [DEBUG] <MainProcess> (MainThread) Loading model from cache C:\Users\tracy\AppData\Local\Temp\jieba.cache
Loading model cost 0.582 seconds.
2018-10-01 21:10:18,274 [DEBUG] <MainProcess> (MainThread) Loading model cost 0.582 seconds.
Prefix dict has been built succesfully.
2018-10-01 21:10:18,274 [DEBUG] <MainProcess> (MainThread) Prefix dict has been built succesfully.


1

In [4]:
# segment train comment
logger.info("start seg train data")
content_train = seg_words(content_train)
logger.info("complete seg train data")

2018-10-01 21:10:25,566 [INFO] <MainProcess> (MainThread) start seg train data
2018-10-01 21:12:44,572 [INFO] <MainProcess> (MainThread) complete seg train data


In [9]:
# segment validation comment
logger.info("start seg validation data")
content_val = seg_words(content_val)
logger.info("complete seg validation data")

2018-10-01 21:44:04,646 [INFO] <MainProcess> (MainThread) start seg validation data
2018-10-01 21:44:25,561 [INFO] <MainProcess> (MainThread) complete seg validation data


In [16]:
# segment validation comment
logger.info("start seg test data")
content_test = seg_words(content_test)
logger.info("complete seg test data")

2018-10-01 21:45:28,086 [INFO] <MainProcess> (MainThread) start seg test data
2018-10-01 21:45:48,098 [INFO] <MainProcess> (MainThread) complete seg test data


In [17]:
content_test[0]

'" 我 想 说 他们 家 的 优惠活动 好 持久 啊 ， 我 预售 的 时候 买 的 券 ， 前两天 心血来潮 去 吃 的 活动 还 在 继续  首先 说 下 服务 ， 因为 和 男票 开车 去 的 ， 有点 不 认路 ， 老板 很 耐心 的 在 电话 里 帮 我们 指路 ， 到 了 门店 之后 也 帮 我们 推荐 了 他们 家 做 的 比较 地道 的 伤心 凉粉 ， 说 是 厨师 是 四川 那边 来 的 。  环境 呢 比较简单 干净 ， 去 的 时候 下午 一点多 了 ， 还有 四五桌 人 在 用餐  口味 对于 我 而言 点 了 麻辣 的 口感 正 正好 ， 男票 比较 能 吃 辣 ， 相对而言 觉得 他们 家 的 麻辣 口感 麻有 了 ， 辣 还 欠缺 一点 ， 老板娘 说 考虑 到 客人 口味 不同 所以 没敢 放太多 辣椒 ， 能 吃 辣 的 朋友 可以 考虑 下单 之前 和 老板 先 说好 。 鱼 呢 我们 选 的 是 黑鱼 ， 2.9 斤 的 鱼 加上 一盆 我 以为 没有 什么 东西 实际上 东西 很多 的 锅底 ， 我们 吃 的 饱饱 的 ， 最后 以为 吃 的 差不多 了 ， 打包 一看 简直 像 没动 过 一样 ， 分量 还是 满足 的 ， 鱼 比较 新鲜 。 伤心 凉粉 很辣 ， 不过 口味 也 蛮 好吃 的 。  总的来说 ， 性价比 还是 可以 的 ， 两个 人 吃 了 大概 160 左右 ， 用 了 团购 券 的话 一百块 不到 ， 会 考虑 下次 再 来 "'

In [18]:
content_test[1]

'" 终于 开 到 心心念念 的 LAB   loft 。 第一次 来 就 随便 点 也 一些 ～ 【 香辣虾 意 面 】 蛮辣 的 ， 但 其实 一般般 。 【 玛格丽特 】 进口 的 感觉 蛮 好 的 就是 喝完 后 就 点 呛 ～ 但是 朋友 不是 很 喜欢 【 一柱擎天 】 看 点评 很多 人 说 喜欢 就 点 了 ， 水蜜桃 味 ， 还 不错 挺 好喝 的 ～ 赞 【 海鲜 饭 】 想 吃饭 但 这店 的 饭类 只有 两种 ， 就 点 了 这个 。 一般般 吧 量 有点 少 ～ 性价比 不高 。 【 奶油 酒 】 这 两杯 是 送 的 ， 应该 是 圣诞 搞 活动 吧 买 半 打 送 半 打 ， 80 块 半 打 ～ 这个 真的 很 好喝 ～ 但 我 和 朋友 就 两个 人想 分别 尝尝 其它 东西 就 没有 点 了 ～ 这个 真心 推荐 ！ "'

## Extract features from corpus

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

logger.info("start train feature extraction")
vector = TfidfVectorizer(analyzer='word', ngram_range=(1, 5), min_df=5, norm='l2', stop_words=stop_words())
x_train = vector.fit_transform(content_train)
logger.info("end train feature extraction")

2018-10-01 22:08:16,867 [INFO] <MainProcess> (MainThread) start train feature extraction
2018-10-01 22:12:00,902 [INFO] <MainProcess> (MainThread) end train feature extraction


In [47]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score

columns = train_data_df.columns.values.tolist()
classifier_dict = dict()
for column in columns[2:]:
    train_data_df[column] = train_data_df[column].astype(int)
    y_train = train_data_df[column]
    clf = MultinomialNB(alpha=.1)
    logger.info("start train %s model" % column)
    clf.fit(x_train, y_train)
    logger.info("complete train %s model" % column)
    classifier_dict[column] = clf

2018-10-01 22:32:00,126 [INFO] <MainProcess> (MainThread) start train location_traffic_convenience model
2018-10-01 22:32:00,307 [INFO] <MainProcess> (MainThread) complete train location_traffic_convenience model
2018-10-01 22:32:00,315 [INFO] <MainProcess> (MainThread) start train location_distance_from_business_district model
2018-10-01 22:32:00,484 [INFO] <MainProcess> (MainThread) complete train location_distance_from_business_district model
2018-10-01 22:32:00,492 [INFO] <MainProcess> (MainThread) start train location_easy_to_find model
2018-10-01 22:32:00,658 [INFO] <MainProcess> (MainThread) complete train location_easy_to_find model
2018-10-01 22:32:00,666 [INFO] <MainProcess> (MainThread) start train service_wait_time model
2018-10-01 22:32:00,836 [INFO] <MainProcess> (MainThread) complete train service_wait_time model
2018-10-01 22:32:00,843 [INFO] <MainProcess> (MainThread) start train service_waiters_attitude model
2018-10-01 22:32:01,009 [INFO] <MainProcess> (MainThread) c

In [51]:
# validation
x_val = vector.transform(content_val)
f1_score_dict = dict()
for column in columns[2:]:
    validate_data_df[column] = validate_data_df[column].astype(int)
    y_val = validate_data_df[column]
    clf = classifier_dict[column]
    pred = clf.predict(x_val)
    f1_score_dict[column] = f1_score(pred, y_val, average='macro')
    
f1_score_dict

c:\users\tracy\anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


{'dish_look': 0.2181846019665296,
 'dish_portion': 0.2092170731883179,
 'dish_recommendation': 0.23921995096490833,
 'dish_taste': 0.24548953352668715,
 'environment_cleaness': 0.2232204273500469,
 'environment_decoration': 0.24974349344032257,
 'environment_noise': 0.22577399571141182,
 'environment_space': 0.21780979190199287,
 'location_distance_from_business_district': 0.22643503535773532,
 'location_easy_to_find': 0.2204911963591554,
 'location_traffic_convenience': 0.2445325322774234,
 'others_overall_experience': 0.24535768940567906,
 'others_willing_to_consume_again': 0.20392012462302886,
 'price_cost_effective': 0.22494195060084002,
 'price_discount': 0.2037467637575387,
 'price_level': 0.22783173959502945,
 'service_parking_convenience': 0.24269149693205386,
 'service_serving_speed': 0.23214885014572625,
 'service_wait_time': 0.23520169077137298,
 'service_waiters_attitude': 0.28248834433919284}

In [28]:
len(content_train)

105000